In [1]:
# import modin.config as cfg
# cfg.Engine.put("dask")
# cfg.CpuCount.put(12) # 12 cores
# cfg.GpuCount.put(1) 

In [2]:
# from distributed import Client
# client = Client(n_workers=cfg.CpuCount.get(), memory_limit="32GB")

In [3]:
import os, gc
import numpy as np
import numba
numba.set_num_threads(12) # 12 = cores

In [4]:
import pandas as pd
pd.set_option('use_numba', True)
# import modin.pandas as pd


In [5]:
import matplotlib.pyplot as plt
from tqdm import tqdm

In [6]:
import time,datetime
from tqdm import tqdm
# from multiprocessing import Pool as ThreadPool

### Config

In [7]:
data_dir = './' # ../input/amex-default-prediction
data_dir_1 = './' # ../input/amex-default-prediction

### Preparation

In [5]:
# # reading csv in chunks
# train_data = pd.DataFrame()
# with pd.read_csv(os.path.join(data_dir, 'train_data.csv'), chunksize=10**5) as reader:
#     for counter, chunk in enumerate(reader):
#         for column in chunk.columns:
#             if str(chunk[column].dtype) == "float64":
#                 chunk[column] = chunk[column].astype(np.float32)
#             if str(chunk[column].dtype) == "int64":
#                 chunk[column] = chunk[column].astype(np.int32)
#         train_data = pd.concat([train_data, chunk])
        


In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5531451 entries, 0 to 5531450
Columns: 190 entries, customer_ID to D_145
dtypes: float32(185), float64(1), int8(2), object(2)
memory usage: 3.9+ GB


In [7]:
train_data['D_64'].unique()

array(['O', 'R', nan, 'U', '-1'], dtype=object)

In [8]:
def denoise(df):
    df['D_63'] = df['D_63'].apply(lambda t: {'CR':0, 'XZ':1, 'XM':2, 'CO':3, 'CL':4, 'XL':5}[t]).astype(np.int8)
    df['D_64'] = df['D_64'].apply(lambda t: {np.nan:-1, 'O':0, '-1':1, 'R':2, 'U':3}[t]).astype(np.int8)
#     df['D_64'] = df['D_64'].apply(lambda t: {None:-1, 'O':0, '-1':1, 'R':2, 'U':3}[t]).astype(np.int8)    
    for col in tqdm(df.columns):
        if col not in ['customer_ID','S_2','D_63','D_64']:
            df[col] = np.floor(df[col]*100)
            # Return the floor of the input, element-wise. 
            # np.floor(np.array([-1.7, -1.5, -0.2, 0.2, 1.5, 1.7, 2.0])) => array([-2., -2., -1.,  0.,  1.,  1.,  2.])
    return df


In [9]:
train_data = denoise(train_data)

100%|██████████| 190/190 [00:04<00:00, 45.86it/s]


In [10]:
train_data.to_parquet("train_data",  compression='gzip', index=False)

In [1]:
del train_data

In [15]:
# reading csv in chunks
# test_data = pd.DataFrame()
# with pd.read_csv(os.path.join(data_dir, 'test_data.csv'), chunksize=10**5) as reader:
#     for counter, chunk in enumerate(reader):
#         for column in chunk.columns:
#             if str(chunk[column].dtype) == "float64":
#                 chunk[column] = chunk[column].astype(np.float32)
#             if str(chunk[column].dtype) == "int64":
#                 chunk[column] = chunk[column].astype(np.int32)
#         test_data = pd.concat([test_data, chunk])

In [18]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11363762 entries, 0 to 11363761
Columns: 190 entries, customer_ID to D_145
dtypes: float32(185), float64(1), int8(2), object(2)
memory usage: 8.1+ GB


In [17]:
test_data = denoise(test_data)

100%|██████████| 190/190 [00:07<00:00, 24.01it/s]


In [19]:
test_data.to_parquet("test_data",  compression='gzip', index=False)

In [12]:
del test_data

### Parquet section

In [5]:
# train_data = pd.read_parquet(os.path.join(data_dir_1, 'train_data'))

In [8]:
# test_data = pd.read_parquet(os.path.join(data_dir_1, 'test_data'))

In [ ]:
df_tr = pd.read_parquet(os.path.join(data_dir_1, 'train_data'))
df_te = pd.read_parquet(os.path.join(data_dir_1, 'test_data'))
df = pd.concat([df_tr,df_te], ignore_index=True)
del df_tr, df_te
_ = gc.collect()
df = df.reset_index(drop=True)
## df = df._to_pandas() # convert modin dataframe to pandas.
# df.to_parquet(f'train_test_data', compression='gzip', index=False)
# del df
# _ = gc.collect()

In [10]:
df.info()

<class 'modin.pandas.dataframe.DataFrame'>
RangeIndex: 16895213 entries, 0 to 16895212
Columns: 190 entries, customer_ID to D_145
dtypes: float32(185), object(2), int8(2), float64(1)
memory usage: 12.1 GB


In [9]:
def one_hot_encoding(df,cols,is_drop=True):
    # cols => ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
    for col in cols:
        # print('one hot encoding:',col)
            # col, df[col].unique() => B_30, [  0. 200. 100.]
        dummies = pd.get_dummies(pd.Series(df[col]),prefix='oneHot_%s'%col)        
            # dummies.columns => Index(['oneHot_B_30_0.0', 'oneHot_B_30_100.0', 'oneHot_B_30_200.0'], dtype='object')
            # dummies['oneHot_B_30_0.0'].unique() => [1 0]
        df = pd.concat([df,dummies],axis=1)
    if is_drop:
        df.drop(cols, axis=1, inplace=True)
    return df

def cat_feature(df):
    # type(df) => <class 'pandas.core.frame.DataFrame'>
    one_hot_features = [col for col in df.columns if 'oneHot' in col]  
    if lastk is None:
        num_agg_df = df.groupby("customer_ID",sort=False)[one_hot_features].agg(['mean', 'std', 'sum', 'last'])
        # sort=False; as customer_IDs are alphanumeric.
        # num_agg_df => dataframe with columns (['mean', 'std', 'sum', 'last']) => 
        # MultiIndex([('oneHot_B_30_0.0', 'mean'),
        #             ('oneHot_B_30_0.0', 'std'),
        #             ('oneHot_B_30_0.0', 'sum'),
        #             ('oneHot_B_30_0.0', 'last')],
        #             ('oneHot_B_30_100.0', 'mean')],
        #             ...
        #             ...
        #            )
        #  and rows => unique customer ids

    else:
        num_agg_df = df.groupby("customer_ID",sort=False)[one_hot_features].agg(['mean', 'std', 'sum'])     
    # converting dataframe from 'MultiIndex' columns to 'Index' columns. 
    num_agg_df.columns = ['_'.join(x) for x in num_agg_df.columns]
        # num_agg_df.columns =>
        # Index(['oneHot_B_30_0.0_mean', 'oneHot_B_30_0.0_std', 'oneHot_B_30_0.0_sum',
        #        'oneHot_B_30_0.0_last',        

    if lastk is None:
        # cat_features  => ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
        cat_agg_df = df.groupby("customer_ID",sort=False)[cat_features].agg(['last', 'nunique']) 
        # nunique => Return counts of unique elements in category for a particular customer_ID.
    else:
        cat_agg_df = df.groupby("customer_ID",sort=False)[cat_features].agg(['nunique'])
    cat_agg_df.columns = ['_'.join(x) for x in cat_agg_df.columns]
        # cat_agg_df.columns =>
        # Index(['B_30_last', 'B_30_nunique', 'B_38_last', 'B_38_nunique', 'D_114_last',
        #         ...
        #         ...
        #        'D_68_nunique'],
        #       dtype='object')    
        
    return num_agg_df
    count_agg_df = df.groupby("customer_ID",sort=False)[['S_2']].agg(['count'])
        # S_2 (timestamp_column)
    count_agg_df.columns = ['_'.join(x) for x in count_agg_df.columns]
        # count_agg_df.columns => Index(['S_2_count'], dtype='object')    
    
    df = pd.concat([num_agg_df, cat_agg_df, count_agg_df], axis=1).reset_index()
    print('cat feature shape after engineering', df.shape )

    return df

def num_feature(df):
    if num_features[0][:5] == 'rank_': 
        # num_features => ['rank_P_2', 'rank_D_39', 'rank_B_1', 'rank_B_2', 'rank_R_1', 'rank_S_3',......, 'rank_D_144', 'rank_D_145']        
        num_agg_df = df.groupby("customer_ID",sort=False)[num_features].agg(['last'])
        # .agg(['last']) => keep last rank of each group.

    else:
        # num_features => ['P_2', 'D_39', 'B_1', 'B_2', 'R_1', 'S_3',......, 'D_144', 'D_145']
        # num_features[0], num_features[0][:5] => P_2, P_2         
        if lastk is None:
            num_agg_df = df.groupby("customer_ID",sort=False)[num_features].agg(['mean', 'std', 'min', 'max', 'sum', 'last'])
        else:
            num_agg_df = df.groupby("customer_ID",sort=False)[num_features].agg(['mean', 'std', 'min', 'max', 'sum'])
    num_agg_df.columns = ['_'.join(x) for x in num_agg_df.columns]

    if num_features[0][:5] != 'rank_':
        for col in num_agg_df.columns:
            num_agg_df[col] = num_agg_df[col] // 0.01 # e.g., mean 92.846153 convert 9284.0
            # 92.846153 // 0.01 => 9284.0, 92.846153 / 0.01 => 9284.6153
            
    df = num_agg_df.reset_index()
    print('num feature shape after engineering', df.shape )

    return df

def diff_feature(df):
    diff_num_features = [f'diff_{col}' for col in num_features]
    cids = df['customer_ID'].values
    df = df.groupby('customer_ID')[num_features].diff().add_prefix('diff_')
        # .diff() => difference of a DataFrame element compared with another element in the 'previous row' (default). 
        # before and after no change in row count.
        # .add_prefix() => column labels are prefixed.        
    df.insert(0,'customer_ID',cids)
    if lastk is None:
        num_agg_df = df.groupby("customer_ID",sort=False)[diff_num_features].agg(['mean', 'std', 'min', 'max', 'sum', 'last'])
    else:
        num_agg_df = df.groupby("customer_ID",sort=False)[diff_num_features].agg(['mean', 'std', 'min', 'max', 'sum'])
    num_agg_df.columns = ['_'.join(x) for x in num_agg_df.columns]
    for col in num_agg_df.columns:
        num_agg_df[col] = num_agg_df[col] // 0.01

    df = num_agg_df.reset_index()
    print('diff feature shape after engineering', df.shape )

    return df



In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5531451 entries, 0 to 5531450
Columns: 190 entries, customer_ID to D_145
dtypes: float32(185), float64(1), int8(2), object(2)
memory usage: 3.9+ GB


In [11]:
n_cpu = 2 # 16
transform = [['','rank_','ym_rank_'],[''],['']]
# transform = [['rank_','ym_rank_'],[''],['']] # for debug


for li, lastk in enumerate([None,3,6]):
    for prefix in transform[li]:
# #         df = pd.read_feather(f'./input/train.feather').append(pd.read_feather(f'./input/test.feather'), ignore_index=True).reset_index(drop=True)
                # Append rows of other to the end of caller, returning a new object.
        df = pd.read_parquet(os.path.join(data_dir_1, 'train_data')).append(pd.read_parquet(os.path.join(data_dir_1, 'test_data')), ignore_index=True).reset_index(drop=True) 
#         df = pd.read_parquet(os.path.join(data_dir_1, 'train_data')).reset_index(drop=True) # for debugging
            

        all_cols = [c for c in list(df.columns) if c not in ['customer_ID', 'S_2']]
         # leaving 'customer_ID' and 'S_2'
        
        cat_features = ["B_30","B_38","D_114","D_116","D_117","D_120","D_126","D_63","D_64","D_66","D_68"]
        num_features = [col for col in all_cols if col not in cat_features]
          # num_features => ['P_2', 'D_39', 'B_1', 'B_2', 'R_1', ...,'S_17', ..., 'D_142', 'D_143', 'D_144', 'D_145']
          #  D_136, D_137, D_138, D_142 etc. have NaN values.
        
        for col in [col for col in df.columns if 'S_' in col or 'P_' in col]:
            if col != 'S_2':
                df[col] = df[col].fillna(0)

        if lastk is not None:
            prefix = f'last{lastk}_' + prefix
            print('all df shape',df.shape)
            df['rank'] = df.groupby('customer_ID')['S_2'].rank(ascending=False)
                # .rank(ascending=False) => rank such that current date will get 'poor rank' than old date.
                
            df = df.loc[df['rank']<=lastk].reset_index(drop=True) # keep top 3 rank or top 6 rank.
            df = df.drop(['rank'],axis=1)
            print(f'last {lastk} shape',df.shape)

                    
        if prefix == 'rank_':
            cids = df['customer_ID'].values
            # rank elements in each group. before and after no change in row count.
            df = df.groupby('customer_ID')[num_features].rank(method='average', pct=True).add_prefix('rank_')
                # .rank(pct=True) => Computes percentage rank of values i.e., arrange the ranks between '0.' and '1.'.
                # e.g. => elements - 4, 4, 7, 7, 5, 6
                # elements (sort) - 4, 4, 5, 6, 7, 7
                # ranks    - 1, 2, 3, 4, 5, 6
                # ranks (again) - 1.5, 1.5, 3, 4, 5.5, 5.5 => ranks of repeated items is averaged.
                # ranks (pct) - 0.250000, 0.250000, 0.500000, 0.666667, 0.916667, 0.916667
   
            df.insert(0,'customer_ID',cids)
                # .insert(0,'customer_ID',cids) => insert new column (with name 'customer_ID') at start (with values as 'cids').                
            num_features = [f'rank_{col}' for col in num_features]

        if prefix == 'ym_rank_': # ym => year-month
            cids = df['customer_ID'].values
            df['ym'] = df['S_2'].apply(lambda x:x[:7]) # pick year-month from date column 'S_2'.
            df = df.groupby('ym')[num_features].rank(pct=True).add_prefix('ym_rank_')
            num_features = [f'ym_rank_{col}' for col in num_features]
            df.insert(0,'customer_ID',cids)

        if prefix in ['','last3_']:
            df = one_hot_encoding(df,cat_features,False)

        # Series.cumsum() => Return cumulative sum over a DataFrame or Series axis.
        vc = df['customer_ID'].value_counts(sort=False).cumsum()
            # df['customer_ID'].value_counts(sort=False) => 
            # 0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a    13
            # 00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5    13
            # 00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1    13
            
            # df['customer_ID'].value_counts(sort=False).cumsum() => 
            # 0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a         13
            # 00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5         26
            # 00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1         39
            # ...
            
        batch_size = int(np.ceil(len(vc) / n_cpu))        
            # np.ceil(np.array([-1.7, -0.2, 0.2, 1.7, 2.0])) => array([-1., -0.,  1.,  2.,  2.]) 
            
        dfs = []
        start = 0
        # make list of dataframe batches in accordance to cpu cores.
        for i in range(min(n_cpu,int(np.ceil(len(vc) / batch_size)))):            
            vc_ = vc[i*batch_size:(i+1)*batch_size]
                # vc_ =>
                # df['customer_ID'].value_counts(sort=False).cumsum() => 
                # 0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a         13
                # 00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5         26
                # ...
                # fffff1d38b785cef84adeace64f8f83db3a0c31e8d92eaba8b115f71cab04681    5531451

            dfs.append(df[start:vc_[-1]])
                # df[start:vc_[-1]] => df[0:5531451]
            start = vc_[-1]
            
#         pool = ThreadPool(n_cpu)

        if prefix in ['','last3_']:
#             cat_feature_df = pd.concat(pool.map(cat_feature,tqdm(dfs,desc='cat_feature'))).reset_index(drop=True)
#             cat_feature_df.to_feather(f'./input/{prefix}cat_feature.feather')            
            cat_feature_df = pd.concat([cat_feature(dfs[l]) for l in range(len(dfs))]).reset_index(drop=True) 
            cat_feature_df.to_parquet(f'{prefix}cat_feature', compression='gzip', index=False)
                # tqdm(dfs,desc='cat_feature') => desc; add message to tqdm progress.
                # pool.map(function_name, arguments)

        if prefix in ['','last3_','last6_','rank_','ym_rank_']:
#             num_feature_df = pd.concat(pool.map(num_feature,tqdm(dfs,desc='num_feature'))).reset_index(drop=True)
#             num_feature_df.to_feather(f'./input/{prefix}num_feature.feather')
            num_feature_df = pd.concat([num_feature(dfs[l]) for l in range(len(dfs))]).reset_index(drop=True) 
            num_feature_df.to_parquet(f'{prefix}num_feature', compression='gzip', index=False)

        if prefix in ['','last3_']:
#             diff_feature_df = pd.concat(pool.map(diff_feature,tqdm(dfs,desc='diff_feature'))).reset_index(drop=True)
#             diff_feature_df.to_feather(f'./input/{prefix}diff_feature.feather')
            diff_feature_df = pd.concat([diff_feature(dfs[l]) for l in range(len(dfs))]).reset_index(drop=True) 
            diff_feature_df.to_parquet(f'{prefix}diff_feature', compression='gzip', index=False) 

#         pool.close()



In [19]:
# count nan values
df.isna().sum()

customer_ID          0
S_2                  0
P_2                  0
D_39                 0
B_1                  0
                ...   
D_141           101548
D_142          4587043
D_143           101548
D_144            40727
D_145           101548
Length: 190, dtype: int64

In [1]:
# df[df['customer_ID'] == '0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a'][['S_2', 'B_30']]